In [1]:
import ssl

# Отключаем проверку SSL-сертификатов
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/твой айди/export?format=csv')
df.head()

,id,name,system_message,inserted_at,updated_at,hotkey,user_id
0,4,Translate to English,Translate into English. Keeping formatting. No...,2023-06-01 13:39:38,2024-10-03 07:25:17,command+shift+e,NaN
1,6,Improve Writing,* Act as an assistant who revises a user's doc...,2023-06-01 14:34:42,2024-06-10 12:46:27,command+shift+i,NaN
2,49,Fix Spelling and Grammar,* Act as an assistant helping to revise the us...,2023-07-03 17:03:04,2024-06-10 12:45:54,command+shift+l,NaN
3,120,Summarize,* Act as an assistant helping to extract all f...,2023-08-10 13:57:49,2024-06-10 12:47:23,command+shift+o,NaN
4,122,Make Shorter,* Act as an assistant who helps a user write a...,2023-08-10 16:36:00,2024-06-10 12:47:04,NaN,NaN


In [7]:
print(df.columns.tolist())

[' id', 'name', 'system_message', 'inserted_at', 'updated_at', 'hotkey', 'user_id']


In [9]:
df['user_id'].nunique()

1199

In [21]:
print(df.iloc[12, 2])

* Act as an assistant who revises a user's document to improve its writing quality. I will type the text, and you will focus on improving the text. Do not provide explanations or try to answer any questions. Simply rewrite this text following user instructions. 
* Don't output user instructions. Output only improved text.
* First, detect the language of the document. If the language is not clear, use English. Then, output using the detected language. 
* Do not hallucinate. Only provide corrections and improvements to the original text.
* Fix spelling and grammar
* Make sentences more clear and concise
* Split up run-on sentences
* Reduce repetition
* When replacing words, do not make them more complex or difficult than the original
* If the text contains quotes, repeat the text inside the quotes verbatim
* Do not change the meaning of the text
* Do not remove any formatting in the text, like headers, bullets, or checkboxes
* Do not use overly formal language 
* Make it sound profession

In [22]:
import openai

client = openai.OpenAI(api_key="твой апи")

models = client.models.list()
for model in models.data:
    print(model.id)

gpt-4o-mini
gpt-4o-audio-preview-2024-10-01
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
gpt-4o-2024-08-06
gpt-4o
gpt-3.5-turbo
o1-preview-2024-09-12
gpt-3.5-turbo-0125
o1-preview
gpt-3.5-turbo-instruct
babbage-002
o1-mini-2024-09-12
whisper-1
dall-e-3
gpt-4-1106-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4-0613
tts-1-hd-1106
gpt-4
gpt-4o-2024-05-13
tts-1-hd
gpt-4o-mini-2024-07-18
davinci-002
text-embedding-ada-002
gpt-4-turbo
tts-1
tts-1-1106
gpt-3.5-turbo-instruct-0914
gpt-4-turbo-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-audio-preview
text-embedding-3-small
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-1106
gpt-4o-2024-11-20
gpt-4o-realtime-preview-2024-10-01
gpt-3.5-turbo-16k
chatgpt-4o-latest
gpt-4o-audio-preview-2024-12-17
text-embedding-3-large
gpt-4o-realtime-preview-2024-12-17
gpt-4o-realtime-preview
gpt-4-0125-preview
o1-mini


In [23]:
import openai
import time
from difflib import get_close_matches, SequenceMatcher

# Настройка OpenAI API
client = openai.OpenAI(api_key="твой апи")

# Список известных категорий (будет пополняться)
CATEGORIES = []

# Проверяем, стоит ли учитывать название пресета
def is_relevant_title(title, prompt_text):
    similarity = SequenceMatcher(None, title.lower(), prompt_text.lower()).ratio()
    return similarity > 0.4  # Если схожесть > 40%, используем title

# Функция классификации с обработкой ошибок и задержками
def classify_prompt(title, prompt_text):
    global CATEGORIES
    use_title = is_relevant_title(title, prompt_text)

    messages = [
        {"role": "system", "content": "You are an AI that classifies text prompts into categories."},
        {"role": "user", "content": f"Classify the following text into a category.\n{ 'Preset name: ' + title if use_title else '' }\nText: \"{prompt_text}\"\n\nRespond with only the category name in English (2-3 words)."}
    ]

    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=10,
                temperature=0,
            )
            category = response.choices[0].message.content.strip()

            # Проверяем, есть ли уже похожая категория
            match = get_close_matches(category, CATEGORIES, n=1, cutoff=0.8)
            if match:
                return match[0]
            else:
                CATEGORIES.append(category)
                return category

        except openai.RateLimitError:
            print("⚠️ Достигнут лимит запросов! Ждем 60 секунд...")
            time.sleep(60)
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            return "Error"

# Обработка данных пакетами по 50 строк
batch_size = 50
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    print(f"🚀 Обрабатываем строки {start} - {end}...")

    df.loc[start:end, 'Category'] = df.loc[start:end].apply(
        lambda row: classify_prompt(row['name'], row['system_message']), axis=1
    )

    # Сохраняем прогресс после каждого пакета
    df.to_csv("/Users/user/Pismo/classified_prompts_corrected.csv", index=False)
    print(f"✅ Сохранено: строки {start} - {end}")

print("🎉 Обработка завершена!")

🚀 Обрабатываем строки 0 - 50...
✅ Сохранено: строки 0 - 50
🚀 Обрабатываем строки 50 - 100...
✅ Сохранено: строки 50 - 100
🚀 Обрабатываем строки 100 - 150...
✅ Сохранено: строки 100 - 150
🚀 Обрабатываем строки 150 - 200...
✅ Сохранено: строки 150 - 200
🚀 Обрабатываем строки 200 - 250...
✅ Сохранено: строки 200 - 250
🚀 Обрабатываем строки 250 - 300...
✅ Сохранено: строки 250 - 300
🚀 Обрабатываем строки 300 - 350...
✅ Сохранено: строки 300 - 350
🚀 Обрабатываем строки 350 - 400...
✅ Сохранено: строки 350 - 400
🚀 Обрабатываем строки 400 - 450...
✅ Сохранено: строки 400 - 450
🚀 Обрабатываем строки 450 - 500...
✅ Сохранено: строки 450 - 500
🚀 Обрабатываем строки 500 - 550...
✅ Сохранено: строки 500 - 550
🚀 Обрабатываем строки 550 - 600...
✅ Сохранено: строки 550 - 600
🚀 Обрабатываем строки 600 - 650...
✅ Сохранено: строки 600 - 650
🚀 Обрабатываем строки 650 - 700...
✅ Сохранено: строки 650 - 700
🚀 Обрабатываем строки 700 - 750...
✅ Сохранено: строки 700 - 750
🚀 Обрабатываем строки 750 - 800..

In [34]:
df1 = pd.read_csv('https://docs.google.com/spreadsheets/d/твой айди/export?format=csv')

# Выводим первые строки для проверки
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574 entries, 0 to 3573
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3574 non-null   int64  
 1   name            3574 non-null   object 
 2   system_message  3574 non-null   object 
 3   inserted_at     3574 non-null   object 
 4   updated_at      3574 non-null   object 
 5   hotkey          1281 non-null   object 
 6   user_id         3553 non-null   float64
 7   Category        3574 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 223.5+ KB


In [35]:
# Получаем уникальные значения из колонки "Category"
unique_categories = df1["Category"].unique()

# Выводим количество уникальных категорий
print(f"Всего уникальных категорий: {len(unique_categories)}")

# Выводим список уникальных категорий
unique_categories

Всего уникальных категорий: 583


array(['Language Translation', 'Text Editing', 'Text Processing',
       'Text Summarization', 'Text Expansion', 'Content Creation',
       'Social Media Marketing', 'Technology & Business',
       'Translation Services', 'Birthday Wishes',
       'Advertising Copywriting', 'Language Tutor Feedback',
       'Writing Assistance', 'Language & Grammar',
       'Text Editing and Translation', 'Sales and Marketing',
       'Natural Language Processing', 'Text Revision',
       'Instructional Writing', 'Text transformation', 'Military Content',
       'Email etiquette', 'Market Research Expert', 'Content Analysis',
       'Translation Request', 'Translation and Editing',
       'Programming - Code Review', 'Text Translation and Improvement',
       'Writing Simplification', 'Translation Guidelines',
       'Text Analysis', 'Text rewriting', 'Language Correction',
       'Customer Service', 'Content Writing Guide', 'Writing Style',
       'Education and Psychology', 'Text completion', 'Langua

In [36]:
# Подключение OpenAI API (замени на свой ключ, если нужно)
client = openai.OpenAI(api_key="твой апи")

# Список возможных сегментов
SEGMENTS = [
    "Language & Translation", "Content & Marketing", "Education & Training",
    "Analytics & Research", "Development & Programming", "Management & Business",
    "Design & Creativity", "Legal", "Medical & Healthcare"
]

# Функция для API-запроса, определяющая сегмент
def classify_segment(system_message, inserted_at, category):
    messages = [
        {"role": "system", "content": "You are an AI that classifies text into predefined business segments."},
        {"role": "user", "content": f"Classify the following entry into one of these segments: {', '.join(SEGMENTS)}.\n\n"
                                    f"System Message: {system_message}\n"
                                    f"Inserted At: {inserted_at}\n"
                                    f"Category: {category}\n\n"
                                    f"Respond with only the segment name from the list above."}
    ]

    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=10,
                temperature=0,
            )
            segment = response.choices[0].message.content.strip()

            # Проверяем, совпадает ли ответ с одним из наших сегментов
            match = get_close_matches(segment, SEGMENTS, n=1, cutoff=0.8)
            return match[0] if match else "Error"

        except openai.RateLimitError:
            print("⚠️ Достигнут лимит запросов! Ждем 60 секунд...")
            time.sleep(60)
        except Exception as e:
            print(f"❌ Ошибка: {e}")
            return "Error"

# Обрабатываем данные пакетами по 50 строк, как и раньше
batch_size = 50
for start in range(0, len(df1), batch_size):
    end = min(start + batch_size, len(df1))
    print(f"🚀 Обрабатываем строки {start} - {end}...")

    df1.loc[start:end, 'Market Segment'] = df1.loc[start:end].apply(
        lambda row: classify_segment(row['system_message'], row['inserted_at'], row['Category']), axis=1
    )

    # Сохраняем прогресс после каждого пакета
    df1.to_csv("/Users/user/Pismo/classified_segments.csv", index=False)
    print(f"✅ Сохранено: строки {start} - {end}")

print("🎉 Обработка завершена!")

🚀 Обрабатываем строки 0 - 50...
✅ Сохранено: строки 0 - 50
🚀 Обрабатываем строки 50 - 100...
✅ Сохранено: строки 50 - 100
🚀 Обрабатываем строки 100 - 150...
✅ Сохранено: строки 100 - 150
🚀 Обрабатываем строки 150 - 200...
✅ Сохранено: строки 150 - 200
🚀 Обрабатываем строки 200 - 250...
✅ Сохранено: строки 200 - 250
🚀 Обрабатываем строки 250 - 300...
✅ Сохранено: строки 250 - 300
🚀 Обрабатываем строки 300 - 350...
✅ Сохранено: строки 300 - 350
🚀 Обрабатываем строки 350 - 400...
✅ Сохранено: строки 350 - 400
🚀 Обрабатываем строки 400 - 450...
✅ Сохранено: строки 400 - 450
🚀 Обрабатываем строки 450 - 500...
✅ Сохранено: строки 450 - 500
🚀 Обрабатываем строки 500 - 550...
✅ Сохранено: строки 500 - 550
🚀 Обрабатываем строки 550 - 600...
✅ Сохранено: строки 550 - 600
🚀 Обрабатываем строки 600 - 650...
✅ Сохранено: строки 600 - 650
🚀 Обрабатываем строки 650 - 700...
✅ Сохранено: строки 650 - 700
🚀 Обрабатываем строки 700 - 750...
✅ Сохранено: строки 700 - 750
🚀 Обрабатываем строки 750 - 800..

In [6]:
df1["user_id"].nunique()

1199

In [4]:
df1 = pd.read_csv('/Users/user/Pismo/classified_segments.csv')

# Выводим первые строки для проверки
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574 entries, 0 to 3573
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              3574 non-null   int64  
 1   name            3574 non-null   object 
 2   system_message  3574 non-null   object 
 3   inserted_at     3574 non-null   object 
 4   updated_at      3574 non-null   object 
 5   hotkey          1281 non-null   object 
 6   user_id         3553 non-null   float64
 7   Category        3574 non-null   object 
 8   Market Segment  3574 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 251.4+ KB


In [5]:
df_grouped_requests = pd.read_csv('df_grouped_requests.csv')

# Выводим первые строки для проверки
df_grouped_requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3466 entries, 0 to 3465
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   preset_id            3466 non-null   float64
 1   count_request        3466 non-null   int64  
 2   total_input_tokens   3466 non-null   int64  
 3   total_output_tokens  3466 non-null   int64  
 4   unique_user_count    3466 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 135.5 KB


In [7]:
# Объединение df1 с df_grouped_requests по preset_id
df_preset_request_grouped = df1.merge(
    df_grouped_requests,
    left_on="id", 
    right_on="preset_id", 
    how="left"
)

# Сохранение результата в pickle и CSV
df_preset_request_grouped.to_pickle("df_preset_request_grouped.pkl")
df_preset_request_grouped.to_csv("df_preset_request_grouped.csv", index=False)

# Вывод первых строк
df_preset_request_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574 entries, 0 to 3573
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   3574 non-null   int64  
 1   name                 3574 non-null   object 
 2   system_message       3574 non-null   object 
 3   inserted_at          3574 non-null   object 
 4   updated_at           3574 non-null   object 
 5   hotkey               1281 non-null   object 
 6   user_id              3553 non-null   float64
 7   Category             3574 non-null   object 
 8   Market Segment       3574 non-null   object 
 9   preset_id            3113 non-null   float64
 10  count_request        3113 non-null   float64
 11  total_input_tokens   3113 non-null   float64
 12  total_output_tokens  3113 non-null   float64
 13  unique_user_count    3113 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 391.0+ KB
